[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/khigashi1987/scanpy_handson_utokyolec2025/blob/main/Python_scRNAseq_Colab.ipynb)

In [ ]:
!pip install fastapi==0.103.2
!pip install --quiet scvi-colab
from scvi_colab import install
install()

In [ ]:
!pip install scanpy scikit-misc python-igraph leidenalg

ここでランタイムの再起動！

In [ ]:
!git clone https://github.com/khigashi1987/scanpy_handson_utokyolec2025.git
%cd scanpy_handson_utokyolec2025

# PythonによるscRNA-seq解析

## 使用データ

マウス15.5日胚の眼球から取り出した網膜の細胞。10X Chromium 3′ v2で解析。2つのReplicatesでそれぞれ約2,600細胞。

Mouse developing retina from Lo Giudice et al. [https://doi.org/10.1242/dev.178103](https://doi.org/10.1242/dev.178103)

Lo Giudice, Q., Leleu, M., La Manno, G. & Fabre, P. J. Single-cell transcriptional logic of cell-fate specification and axon guidance in early-born retinal neurons. Development 146, dev178103 (2019).

## Pythonに関係ない部分の解析

### データのダウンロード

好きな方法で公共データベースからダウンロードする。レプリケイトがふたつあって、それぞれ以下のSRR IDに対応。
```bash
fastq-dump SRR8181428 --split-files --gzip
fastq-dump SRR8181429 --split-files --gzip
```

SRR8181428       ([Rub browser](https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR8181428&display=metadata))([メタデータ](https://ncbi.nlm.nih.gov/sra/SRR8181428) )  
```bash
SRR8181428_1.fastq.gz ：Read 1  
SRR8181428_2.fastq.gz ：Read 2  
SRR8181428_3.fastq.gz ：Index  
```
SRR8181429       ([Rub browser](https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR8181429&display=metadata))([メタデータ](https://ncbi.nlm.nih.gov/sra/SRR8181429) )  
```bash
SRR8181428_1.fastq.gz ：Read 1  
SRR8181428_2.fastq.gz ：Read 2  
SRR8181428_3.fastq.gz ：Index 
```
Cell Ranger は、以下の命名ルールに沿った fastq file 名が必要
```bash
'[サンプル名]_S[サンプル番号]_L00[レーン番号]_[リードタイプ]_001.fastq.gz'
```
今回は、以下のようにファイル名を変更した。
```bash
Read 1: SRR8181428_S1_L001_R1_001.fastq.gz
Read 2: SRR8181428_S1_L001_R2_001.fastq.gz
Index : SRR8181428_S1_L001_I1_001.fastq.gz
```
```bash
Read 1: SRR8181429_S1_L001_R1_001.fastq.gz
Read 2: SRR8181429_S1_L001_R1_001.fastq.gz
Index : SRR8181429_S1_L001_I1_001.fastq.gz
```

### Cell Rangerによる解析

Cell Ranger v.9.0.0でマウスゲノム（mm10）へのマッピング、遺伝子ごとのカウントを実行。

レプリケイトは（論文でそう名付けられてたので）E2, F2という名前のバッチに設定。

なお、最新版のcellrangerでは "--create-bam" オプションが必須になっている点に注意。マッピング結果のBAM形式ファイルを利用する予定がなければ、計算量とディスク節約のためにfalseに設定すれば良いが、ツールによって（後半で扱うvelocytoなど）BAMファイルが必須の場合がある。

```bash
cellranger count --id=RetinalBatchE2 \
   --fastqs=/path/to/SRR8181428\
   --sample=SRR8181428 \
   --create-bam=true \
   --transcriptome=/path/to/Cell_Ranger_References/refdata-gex-mm10-2020-A

cellranger count --id=RetinalBatchF2 \
   --fastqs=/path/to/SRR8181429\
   --sample=SRR8181429 \
   --create-bam=true \
   --transcriptome=/path/to/Cell_Ranger_References/refdata-gex-mm10-2020-A
```

講習では、このCell Ranger解析の結果ディレクトリ（の一部、講習で使う部分だけを抜き出したもの）を配布している。

こんな感じの構成になっていて、Cell Rangerのアウトプットが見えるはず。

./data の中身  

     data -- RetinalBatchE2 - outs - filtered_feature_bc_matrix - barcodes.tsv.gz  # cell index  
                                                                - features.tsv.gz  # 遺伝子 Id  
                                                                 - matrix.mtx.gz    # count データ  

          -- RetinalBatchF2 - outs - filtered_feature_bc_matrix - barcodes.tsv.gz  # cell index
                                                                - features.tsv.gz  # 遺伝子 Id
                                                                - matrix.mtx.gz    # count データ

                                                          

## ライブラリのimport

定番のライブラリのimportに加えて、scanpyをscの別名でimportする。

Figのちょっとした設定。各ライブラリのバージョンは以下の通り。(インストールした際の python のバージョンなどで ライブラリのバージョンが異なっている可能性はあるかもしれません。)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

## データの読み込み

Scanpyの特徴は、データフレームを拡張した ***anndata*** と呼ばれる オブジェクトを使う点にある。anndataを使うと、ひとつのオブジェクトに遺伝子発現量のデータ、サンプルや細胞のアノテーション、遺伝子の情報などをまとめて格納できる。 anndataを使うことで、実験の情報が詰まったひとつのオブジェクトに対して処理を次々に実行し、さらに処理結果をそこに蓄積していくことができる。

Scanpyでは、10x Genomicsのデータは、結果のディレクトリを指定することでそのままロードが可能な関数が用意されている。

ディレクトリには、3つのファイルが書き出されている。  

**barcodes.tsv:** 個別の細胞を識別するバーコードが記述されたテキストファイル。各行ごとにひとつのバーコードが記述されている。  
**genes.tsv:** 計測された遺伝子が記述されたタブ区切りテキストファイル。1列目に遺伝子のENSEMBL Gene ID、2列目の遺伝子のシンボルが記述されている。  
**matrix.mtx:** 各細胞（バーコード）について各遺伝子の発現がいくつ観測されたかのカウント情報をまとめたファイル。このファイルはMatrix Market Exchange Formatsという形式で記述されており、疎行列（含まれる値の多くがゼロであるような行列）を比較的コンパクトに記述するための形式となっている。  


Scanpyの10xデータ用読み込み関数を使うと、これらを同時に読み込んで、適切に構造化されたオブジェクトができあがる。

In [ ]:
adata_E2 = sc.read_10x_mtx(path='./data/RetinalBatchE2/outs/filtered_feature_bc_matrix/')
adata_F2 = sc.read_10x_mtx(path='./data/RetinalBatchF2/outs/filtered_feature_bc_matrix/')

In [ ]:
adata_E2

3392 細胞 × 32285 遺伝子

In [ ]:
adata_F2

3611 細胞 × 32285 遺伝子

2つのバッチをひとつのanndataオブジェクトに統合する。バッチのラベルもここで設定。

In [ ]:
adata = adata_E2.concatenate(adata_F2, batch_categories=['E2', 'F2'])

観測値（細胞）に関するデータは、**obs** でアクセスする。 observationsの略。このデータフレームに細胞に関するデータを追加していくことができる。

In [ ]:
adata.obs

変数（遺伝子）に関するデータは、**var** でアクセスする。variable の略。現在は、遺伝子のシンボル（名前）と、Ensembl Gene IDなどが登録されている。

In [ ]:
adata.var

カウントテーブルには、以下のように ***X*** でアクセスできる。疎行列として格納されている。

In [ ]:
adata.X

In [ ]:
adata

E2: 3611 細胞 × 32285 遺伝子  
F2: 3392 細胞 × 32285 遺伝子 を足したデータ 7003 細胞 × 32285 遺伝子 が格納されている。

## 前処理

観測された遺伝子が極端に少ない細胞、割り当てられた細胞が極端に少ない遺伝子をフィルタリング。

In [ ]:
# 観測された遺伝子が極端に少ない細胞
sc.pp.filter_cells(adata, min_genes=200)
# 割り当てられた細胞が極端に少ない遺伝子
sc.pp.filter_genes(adata, min_cells=20)

In [ ]:
adata

7003 細胞 × 32285 遺伝子  -> 7001 細胞 × 15001 遺伝子

ミトコンドリア遺伝子の発現割合でフィルタリングする予定なので、遺伝子側のメタデータにミトコンドリア遺伝子か否かの情報を付与。

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')

In [ ]:
adata.var['mt']

`sc.pp.calculate_qc_metrics` は、クオリティコントロールに関連したいくつかの指標を一挙に計算してくれる便利な関数。

`qc_vars`に遺伝子側のメタデータラベルを設定すると、それについてもカウントとパーセンテージを勝手に計算してくれる。

In [ ]:
# inplace オプション: 計算した結果を .obs と .var に書き込むか？
# percent_top オプション: Which proportions of top genes to cover. If empty or None don’t calculate. Values are considered 1-indexed, percent_top=[50] finds cumulative proportion to the 50th most expressed gene.
# log1p オプション: Set to False to skip computing log1p transformed annotations.
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=False, log1p=False, inplace=True) 

それぞれの分布をプロット。

In [ ]:
sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)

#n_genes_by_counts: 各細胞ごとにカウントがあった遺伝子数   #total_counts: 各細胞ごとの合計カウント数   #pct_mt_counts: ミトコンドリアのカウントのパーセント  

In [ ]:
sc.pl.scatter(adata, 'total_counts', 'n_genes_by_counts', color='pct_counts_mt', size=40)

In [ ]:
fig = plt.figure()
sns.displot(adata.obs['pct_counts_mt'], kde=False)
plt.show()

In [ ]:
fig = plt.figure()
sns.displot(adata.obs['pct_counts_mt'][adata.obs['pct_counts_mt'] < 10], kde=False)
plt.show()

分布を見て方針を決めて、フィルタリングを実行。

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 2000)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = 13000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 1000)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['pct_counts_mt'] < 6]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

In [ ]:
adata

7003 細胞 × 32285 遺伝子 -> 7001 細胞 × 15001 遺伝子 -> 4773 細胞 × 15001 遺伝子

正規化や対数変換などの前に、この段階のカウントマトリックスを別のレイヤーに退避させておく。あとで確率モデル推定のときに必要になる。レイヤーに保管したデータは基本的に以降の操作の影響を受けないが、細胞や遺伝子のslicingは適用されるので注意。（常にマトリックスのshapeは一致する）

In [ ]:
adata.layers['counts'] = adata.X.copy()

## 正規化

ライブラリサイズによる正規化、対数変換などの前処理は、scanpy.pp以下にいくつか便利な関数がある。

ここでは、細胞ごとのカウントの和が10,000になるように正規化してから、対数変換。

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)

この段階のデータ（いろいろとややこしい変換をしていない「ピュア」なデータ）も、あとでプロットや結果解釈のときに使いたいので退避させておく。rawは特別なレイヤーで、数値だけでなくobsやvarのメタデータも含めてフリーズしてくれる。また遺伝子側のsliceの影響を受けない（細胞側のsliceは適用される）

In [ ]:
adata.raw = adata

## 特徴量選択（発現量の変動が大きい遺伝子）

発現量変動の大きい遺伝子のみを抽出して、データのサイズを小さくする。

発現量変動の大きい遺伝子とは、細胞間の発現量の分散が指定した値以上に大きい遺伝子を取得すればいいが、遺伝子の平均発現量毎に分散も異なるので、正規化分散をしてから特徴量選択をする。
seurat, cell_ranger, seurat_v3 のアルゴリズムが選択できる。 本講習ではデフォルトの seurat を使用する。

In [ ]:
sc.pp.highly_variable_genes?

In [ ]:
# top 2000genes のみを抽出する
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat')
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
sc.pl.highly_variable_genes(adata)

計算結果は自動的に adata.var 、つまり、遺伝子に関するメタデータを格納したオブジェクトに追加される。 highly_variable の項目が True の遺伝子が高発現変動遺伝子。

In [ ]:
adata.var

## 次元削減

主成分分析 (PCA) は、データ点全体の分布の大局的 (グローバル) な構造をよりよく表現する。
それに対して、t-SNE, UMAP は、どちらかというと局所的 (ローカル) な構造を重視する。高次元空間で近くに配置されている点を低次元空間でも近くに配置していく。その結果、どの細胞とどの細胞が似ているか、似ている細胞集団から構成されるクラスタがいくつあるのかといった生物学的な解釈に結びつけやすいので、scRNA-seq解析でよく使われている。しかしながら、クラスタ間の距離関係、このクラスターは、あっちのクラスターとどのくらい離れている、、とかはわからない。クラスタ間の距離の比較はできない。あくまで隣接している細胞同士の距離関係が最適化するように学習されている。

 t-SNE のアルゴリズム  
 高次元空間と低次元空間の距離関係を一致させて、どうやって比較するか？
  1. 低次元側のデータ点の座標をランダムに (あるいは主成分分析やなんらかの高速な手法で暫定的に) 配置する
  2. 高次元空間上の「距離」と低次元空間上の「距離」を計測するための関数を設定する
  3. 高次元空間上の「距離」と低次元空間上の「距離」を比較して、誤差を表現する (コスト関数) を設定する
  4. コスト関数をなんらかの最適化計算手法で最小化する

 tSNE と比較した UMAP の利点
  1. 計算の実行時間が tSNE より早い (が、実装によっては t-SNE も十分速い)
  2. 大規模なデータセットでも t-SNE ほどメモリを消費せずにすむ
  3. データ全体のグローバルな構造を tSNE よりもよく保存する (と言われている)
  4. アルゴリズムが代数的位相幾何学の強固な理論的基盤に支えられている (?)

　(独習Pythonバイオ情報解析　第11章 シングルセル解析② 次元削減 より)

t-SNE, UMAP ともに、計算効率のために、データの前処理として PCA を行って、特徴量の数を削減したセットを使用する。

## 主成分分析（PCA）

PCAはscanpyの前処理関数で簡単に実行できる。とりあえず、50次元まで落としてみる。 use_highly_variableのフラグをオンにすると、遺伝子全体ではなく前項で決定した高発現変動遺伝子のみを使って次元削減をする。

In [ ]:
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

PCAで落とした座標は、観察値のメタデータを格納する ***obsm*** に自動的に入る。obsm は Multi-dimensional annotation of observations の略。複数の次元でひとつの何かを表現するような観測値のメタデータがここに格納される。

In [ ]:
print(adata.obsm['X_pca'].shape)

7003 細胞 × 32285 遺伝子 -> 7001 細胞 × 15001 遺伝子 -> 4773 細胞 × 15001 遺伝子 -> 4773 細胞 × 50 次元

プロットしてみる。Scanpyでは基本的に「前処理」（Preprocessing）に関わる関数がscanpy.ppに、「プロット」（Plot）に関わる関数がscanpy.plに入っている。

いまのところ特に色つけるようなデータも無いので、適当に細胞ごとのカウントで色分けをした。遺伝子名をcolorに指定するとその遺伝子の発現量によるプロットも可能。

In [ ]:
sc.pl.pca(adata, color='total_counts')

遺伝子 Xkr4 の発現量による色付けの例

In [ ]:
sc.pl.pca(adata, color='Xkr4')

In [ ]:
sc.pl.pca(adata, color='total_counts', 
          components=['1,2', '2,3', '1,3'])

In [ ]:
# projection='3d'
# を指定すると3次元表示（見づらい）
sc.pl.pca(adata, color='total_counts', projection='3d')

In [ ]:
sc.pl.pca(adata, color='batch', projection='3d')

寄与率をグラフで見る

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50)

## t分布型確率的近傍埋め込み（t-SNE）

まず、t-SNE, UMAP共通のステップとして、データから「近傍グラフ」（neighborhood graph）の構築が必要。

In [ ]:
sc.pp.neighbors(adata)

データ点間の接続関係（細胞の近傍関係）は、全細胞vs.全細胞のペアの情報を記録する ***obsp*** （Pairwise annotation of observations の略）に格納される。

In [ ]:
adata.obsp['connectivities']

接続関係を元にしてt-SNEを実行。

In [ ]:
sc.tl.tsne(adata) 

t-SNE結果のプロット。

In [ ]:
sc.pl.tsne(adata, color='total_counts')

## UMAP

近傍グラフはすでに計算しているので、scanpy.tlのumap関数を使えばオーケー。

In [ ]:
sc.tl.umap(adata)

プロットはt-SNEとほとんど同じ。scanpy.pl以下にUMAP用の描画関数がある。

In [ ]:
sc.pl.umap(adata, color='total_counts')

あきらかに似た形状の、ふたつの構造がある。。。

各細胞が由来するバッチで色分けしてみると・・・

In [ ]:
sc.pl.umap(adata, color='batch')

ということで、このふたつの構造はもろにバッチ効果の結果だった。なんらかの方法で補正が必要だが、それは後述。

## クラスタリング

### グラフベースのクラスタリング Leiden クラスタリング
高次空間のデータ点からK近傍グラフ構造を作成して、データ点のクラスタリングを、グラフのコミュニティ検出問題として扱う。グラフ構造から「コミュニティ (点のカタマリ、クラスタ)」を見つける。モジュラリティ:コミュニティへの割り当ての良さの指標。このモジュラリティが最大になるようなコミュニティ割り当てを目指す。

Leidenクラスタリングを実行してみる。モジュラリティの計算に影響を与える "resolution" パラメータが存在する。  
"resolution" は値が大きいほどクラスタが多くなる。default=1

In [ ]:
sc.tl.leiden(adata, resolution=0.5, key_added='leiden_r0.5')

クラスタリングの結果は観測値のメタデータ（obs）の中に格納される。

In [ ]:
adata.obs

プロットする点の座標じたいはUMAPの結果なので、プロットはUMAP版の関数を使い、色分けだけをクラスタリング結果で指定すればいい。

In [ ]:
sc.pl.umap(adata,
           color=['batch', 'leiden_r0.5'],
           ncols=2,
           frameon=False)

## 深層生成モデルの利用

scVIが仮定しているデータの生成プロセスは以下の確率モデルに基づく。  
2023年の講習会動画を参照。(https://www.genome-sci.jp/lecture202212)

全細胞数がN, 細胞のインデックスが $n \in \{1..N\}$、全遺伝子数がG、遺伝子のインデックスが $g \in \{1..G\}$、細胞nのバッチIDが$s_n$であるとしたとき、細胞nの遺伝子gのカウントは、

$$z_n \sim Normal(0, I)$$

$$l_n \sim logNormal(l_\mu, l_\sigma^2)$$

$$\rho_n = f_w (z_n, s_n)$$

$$w_{ng} = Gamma(\rho_n^g, \theta)$$

$$y_{ng} = Poisson(l_n w_{ng})$$

$$h_{ng} = Bernoulli( f_h^g(z_n, s_n) )$$

$$
  \begin{equation}
    x_{ng} =
    \begin{cases}
      y_{ng} & \text{if}\ h_{ng}=0 \\
      0 & \text{otherwise}
    \end{cases}
  \end{equation}
$$

$l_\mu, l_\sigma$は、スケーリングファクターでバッチの数をBとしたとき$l_\mu,l_\sigma \in \mathbb{R}^B$で、実際のバッチごとの平均と分散に設定しておく。

ガンマ分布のパラメータ、ベルヌーイ分布のパラメータは、正規分布からサンプリングした$z_n$を使ってニューラルネットワークで表現する（VAEにおけるreparametrization trick）。

最終的なカウントはガンマとポアソンの複合分布なので、負の二項分布でモデル化してることになる。さらにベルヌーイ分布で、scRNA-seqにありがちなdrop-outイベントを表現。これらの組み合わせで、ゼロ過剰負の二項分布(Zero-inflated negative binomial distribution; ZINB)を表現している。

事後分布はVAEをSGDで最適化して変分推論。

<span style="color: red; ">学習後、$z_n$にアクセスすれば、バッチの効果が除去された細胞の潜在ベクトルが得られるし、$\rho_{ng}$にアクセスすればdrop-outなどのイベントが生じなかった場合の遺伝子発現の期待値が得られる。</span> 

<span style="color: red; ">"drop-out"</span> とは


scRNA-seqでは、各細胞のシーケンスの量が少ないため、特定の遺伝子の発現が検出されないことがある。遺伝子の発現データが欠落していることをドロップアウトという。ドロップアウトが発生すると、データセット内の遺伝子の発現データが不完全になり、遺伝子間の相関や差異が正確に評価されない。この問題を克服するために、統計的手法を用いて欠落したデータを推定 (imputation) または埋めることで、遺伝子発現プロファイルの正確性を向上させる必要がある。

In [ ]:
import scvi

データを格納したanndataオブジェクトと、バッチのIDを指定したobsのカラムのラベルを指定して、モデルをセットアップする。カウントデータの確率モデルなので、（正規化や対数変換したマトリックスではなく）カウントデータを格納したレイヤーを指定する必要があることに注意。

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer='counts',
    batch_key='batch',
)

モデルを作る。ここで中間層のノード数とか潜在ベクトルの次元サイズなど設定できるが、とくに変更しなくていいと思う。

In [ ]:
model = scvi.model.SCVI(adata)

### scVIモデルのトレーニング（バッチ補正）

学習を実行する。GPU使わない場合はめっちゃ時間かかるので今回の講習では割愛。学習済みのモデルパラメータを用意したのでそれをロードして学習できたことにする。  

とはいえ、深層学習としてはたいして規模の大きいデータセットでもないので、iMac（3.6GHz 10コア Intel Core i9, メモリ128GB）でCPUのみでトレーニングしても30分程度。

In [ ]:
#model.train()

In [ ]:
#model.save('./models/scVI_model', overwrite=True)

In [ ]:
model = scvi.model.SCVI.load('./models/scVI_model', adata=adata)

学習し細胞ごとのバッチの効果が除去された細胞の潜在表現$z_n$は以下の関数で取得できる。PCAやUMAPの座標と同じように、obsmに格納しておく。

In [ ]:
adata.obsm['X_scVI'] = model.get_latent_representation()

発現量期待値 $\rho_{ng}$ 、つまりdenoiseされた発現量テーブルは以下の関数で取得できる。適当なライブラリサイズで規格化してくれる。この数値はあとで使いたいので別のレイヤーに格納しておく。

In [ ]:
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size=1e4)

In [ ]:
adata

### scVI潜在表現+UMAPによる次元削減

潜在表現でちゃんとバッチ効果が補正されたかどうか確認してみる。

近傍グラフ計算の `scanpy.pp.neighbors` は、デフォルトではPCAで計算した主成分（adata.obsm['X_pca']）を元にグラフを構築する。ここではPCA結果の座標ではなく、scVIで推定された細胞の潜在ベクトルを指定してグラフ構築してみる。バッチ効果が補正されたグラフが構築されるはず。

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata)
sc.pl.umap(adata, color='batch')

### scVI潜在表現+Leidenによるクラスタリング

同じように、scVI潜在表現で構成された近傍グラフを元にLeidenクラスタリングを実行してみる。上のセルで `scanpy.pp.neighbors` を実行したことにより、adata.obspにはscVI潜在表現で構築された近傍グラフが入っているので、ここでは普通に `scanpy.tl.leiden` を実行すればいい。

In [ ]:
sc.tl.leiden(adata, key_added="leiden_scVI", resolution=1.0)
sc.pl.umap(adata,
           color=['batch', 'leiden_scVI'],
           ncols=2,
           frameon=False)

In [ ]:
# 遺伝子 Top2a, Mcm6の比較でdrop-outが補正されたか確認。
sc.pl.scatter(adata, x='Top2a', y='Mcm6', color='leiden_scVI')

In [ ]:
sc.pl.scatter(adata, x='Top2a', y='Mcm6', use_raw=False, layers='scvi_normalized', color='leiden_scVI')

### Doublet検出

Doublet とは: 2つ以上の細胞が単一細胞としてシーケンスされている状態。

[SOLO](https://doi.org/10.1016/j.cels.2020.05.010)によるDoublet検出。

Bernstein, Nicholas J., et al. "Solo: doublet identification in single-cell RNA-Seq via semi-supervised deep learning." Cell systems 11.1 (2020): 95-101.

時間に余裕があればやる。

In [ ]:
results = []
for batch in ['F2', 'E2']:
    tmp_solo_model = scvi.external.SOLO.from_scvi_model(
        model, restrict_to_batch=batch)
    tmp_solo_model.train()
    result = tmp_solo_model.predict(soft=False)
    # 意図は不明だがSOLO予測のindexになぜか "-0" が付加されるので消しておく
    result.index = result.index.str.replace("-0$", "", regex=True)
    results.append(result)
results = pd.concat(results)

In [ ]:
results[adata.obs.index]

In [ ]:
adata.obs['SOLO_prediction'] = results[adata.obs.index]

In [ ]:
adata.obs['SOLO_prediction'].value_counts()

In [ ]:
sc.pl.umap(adata, color='SOLO_prediction', frameon=False)

### DEG解析

仮説検定ではなく、ベイズ統計の枠組みでモデル比較を行う。基本的にはある集団と別の集団それぞれの遺伝子発現量期待値（VAEでdenoiseした発現量$\rho_{ng}$）のlog fold changeの値が、ある閾値以下となるか、それ以上の変化があるか、のふたつのモデルで比較する。

`groupby`にラベルを指定すると、そのクラス（この場合はleidenクラスタリングのひとつのクラスタ）とその他全部で自動的に比較してくれる。比較のグループを一部に制限したり、ペアを指定したりいろいろと複雑に組み合わせは指定できる。詳細は[APIのドキュメント](https://docs.scvi-tools.org/en/stable/api/reference/scvi.model.SCVI.html#scvi.model.SCVI.differential_expression)に。

In [ ]:
DEs = model.differential_expression(groupby='leiden_scVI', mode="change")
DEs.head()

proba_de: DEG の確率  
proba_not_de: DEG ではない確率  
balyes_factor: ベイズ統計の枠組みでモデル比較した時の指標のひとつ。  
comparison: どのグループとどのグループで比較をした結果か。  

それぞれのクラスタ「特異的」遺伝子、DEG確率の高い順に上から3つくらい見てみる。

In [ ]:
markers = {}
for i, c in enumerate(adata.obs['leiden_scVI'].unique()):
    comparison_label = "{} vs Rest".format(c)
    cluster_df = DEs.loc[DEs['comparison'] == comparison_label]
    cluster_df = cluster_df[cluster_df['lfc_mean'] > 0]  # lfc_mean > 0: そのクラスター特異的に発現している遺伝子
    cluster_df = cluster_df[cluster_df['bayes_factor'] > 3]
    cluster_df = cluster_df[cluster_df['non_zeros_proportion1'] > 0.1]
    markers[c] = cluster_df.index.tolist()[:3]

In [ ]:
markers

図でLeidenクラスタのデンドログラムを表示したいので事前に計算しておく。

In [ ]:
sc.tl.dendrogram(adata, groupby='leiden_scVI', use_rep='X_scVI')

クラスタ間での遺伝子発現量の比較は、`scanpy.pl.dotplot`が見やすくて便利。上で抽出したマーカー遺伝子を指定して、表示する発現量の数値としてはadata.rawに格納したデータで描画する。

In [ ]:
sc.pl.dotplot(
    adata,
    markers,
    groupby='leiden_scVI',
    dendrogram=True,
    color_map="Blues",
    swap_axes=True,
    use_raw=True,
    standard_scale='var')

クラスタ平均値ではなくクラスタごと細胞ごとの全部の（scVIノーマライズされた）発現量でヒートマップを描くこともできる。

In [ ]:
sc.pl.heatmap(
    adata,
    markers,
    groupby='leiden_scVI',
    layer='scvi_normalized',
    standard_scale="var",
    dendrogram=True,
    figsize=(8, 12)
)

## データの保存

In [ ]:
adata.write(filename='./data/retinal_1.h5ad')

## RNA速度の推定

### 事前準備（velocyto）

In [ ]:
!pip install -U scvelo cellrank

scVeloを使うためには、spliced/unsplicedの二種類のカウントデータが必要となる。

なので事前に [velocyto](http://velocyto.org/velocyto.py/index.html) を使って、Cell Rangerが生成したゲノムマッピングのBAMファイルから、spliced/unsplicedのカウントデータを作っておく。

バッチごとに実行して、それぞれのバッチに属する細胞のspliced/unsplicedカウントを格納した [loom](https://linnarssonlab.org/loompy/index.html) データを作る。

計算に時間がかかるので、講習ではすでにvelocyto計算済みの結果を配布している。

velocytoは以下のコマンドで実行する。

```bash
velocyto run10x \
    -m /path/to/Cell_Ranger_References/mm10_rmsk.gtf \
    /path/to/RetinalBatchE2 \
    /path/to/Cell_Ranger_References/refdata-gex-mm10-2020-A/genes/genes.gtf

velocyto run10x \
    -m /path/to/Cell_Ranger_References/mm10_rmsk.gtf \
    /path/to/RetinalBatchF2 \
    /path/to/Cell_Ranger_References/refdata-gex-mm10-2020-A/genes/genes.gtf
```

velocytoでバッチごとに推定したloomファイルを統合して書き出す。ここの統合処理も時間かかるので、統合後のデータを配布。

In [ ]:
#import loompy
#loompy.combine(['./data/RetinalBatchE2.loom',
#                './data/RetinalBatchF2.loom'],
#               output_file='./data/retinal_velo.loom')

In [ ]:
%cd scanpy_handson_utokyolec2025

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

import scvelo as scv
scv.set_figure_params()

adata = sc.read_h5ad('./data/retinal_1.h5ad')

In [ ]:
sc.pl.umap(adata, color=['leiden_scVI'], frameon=False)

spliced/unsplicedのカウントデータを遺伝子発現テーブルと統合する。細胞のバーコードで紐づける。

In [ ]:
ldata = sc.read_loom('./data/retinal_velo.loom')
adata = scv.utils.merge(adata, ldata)

scVeloの関数で、遺伝子ごとのspliced/unsplicedカウントの比率を表示できる。実験プラットフォームにもよるが、だいたいunsplicedが25%程度らしい。クラスタごとにも表示。極端にunsplicedがとれてないクラスタがあるかどうかチェックする。

In [ ]:
scv.pl.proportions(adata, groupby='leiden_scVI')

速度計算を実行する。近傍グラフの構成から。

In [ ]:
# scVI補完の潜在空間上で近傍グラフ構成、一次・二次モーメント計算
scv.pp.moments(adata, use_rep='X_scVI')

Splicing kineticsモデルのパラメータ推論。

"Steady-state"モデル。

In [ ]:
scv.tl.velocity(adata, mode='deterministic')

速度ベクトルを元に、細胞から細胞への遷移確率を計算。

In [ ]:
scv.tl.velocity_graph(adata)

速度ベクトルから構成した「流れ」をプロットしてみる。

In [ ]:
scv.pl.velocity_embedding_stream(adata, 
        basis='X_umap', color='leiden_scVI',
        legend_fontsize=9,
        smooth=0.8, min_mass=4)

Progenitorの集団からはじまって、わかりやすい三分岐。

"EM"モデル。

In [ ]:
# Colabのメモリが不足するためスキップ
#scv.tl.recover_dynamics(adata, n_jobs=8)
#scv.tl.recover_latent_time(
#    adata,
#    root_key='initial_states_probs', 
#    end_key='terminal_states_probs')

「膨らみ方」（splicing kinetics）は遺伝子ごとに異なる。scVeloのscatter関数では遺伝子ごとに、具体的にどのようなダイナミクスが推定されたのかプロットしてくれる。それぞれの遺伝子がそれぞれ異なるタイミング、異なるパターンで、induction => steady-state => repressionのパスを巡っている。

In [ ]:
#top_genes = adata.var["fit_likelihood"].sort_values(ascending=False).index
#scv.pl.scatter(adata, basis=top_genes[:5], color="leiden_scVI", frameon=False)

In [ ]:
#scv.tl.velocity(adata, mode="dynamical")
#scv.tl.velocity_graph(adata, n_jobs=8)

In [ ]:
#scv.pl.velocity_embedding_stream(adata, 
#        basis='X_umap', color='leiden_scVI',
#        legend_fontsize=9,
#        smooth=0.8, min_mass=4)

RNA速度を計算することによって、トランスクリプトームパターンの幾何学的な近接性だけで推定された 'Pseudo time' ではなく、速度情報をちゃんと織り込んだ細胞の 'Latent time' を計算できる。

In [ ]:
#scv.pl.scatter(
#    adata,
#    color=['leiden_scVI', 'latent_time'],
#    fontsize=16,
#    cmap='viridis',
#    perc=[2, 98],
#    colorbar=True,
#    rescale_color=[0, 1],
#    title=['clusters', 'latent time'])

Progenitorの部分がごちゃごちゃしてわかりにくいので、取り出して個別に解析してみる。

どうもグルグルとまわってるみたいなので、細胞周期のマーカー遺伝子発現をチェック。

In [ ]:
prog = adata[adata.obs['leiden_scVI'].isin(['0', '3', '4', '6'])].copy()

In [ ]:
sc.pp.neighbors(prog, use_rep="X_scVI", n_neighbors=3)
sc.tl.umap(prog, min_dist=1.0)

scv.pl.umap(prog,
            color=['leiden_scVI'],
            frameon=False)

In [ ]:
sc.pl.umap(prog, 
           ncols=2,
           color=['Mcm6', 'Esco2', 'Top2a', 'Aurka', 'Cenpa'],
           cmap='viridis',
           frameon=False)

というわけで、このグルグルは細胞周期を反映しているのだった。

この中だけで速度を描くとよりわかりやすい。

In [ ]:
scv.pp.moments(prog, use_rep='X_scVI')
scv.tl.recover_dynamics(prog, n_jobs=8)
scv.tl.velocity(prog, mode="dynamical")
scv.tl.velocity_graph(prog)
scv.pl.velocity_embedding_stream(prog, 
    basis='X_umap', color='leiden_scVI',
    smooth=0.8, min_mass=4)

細かく見ると印象が違ってくることもあるので、流れの全体像として粗視化されたパターンだけでなく、それぞれの速度をつぶさに見ていくことも大事。

In [ ]:
scv.pl.velocity_embedding(adata, figsize=(9, 9),
        basis='X_umap', color='leiden_scVI',
        scale=0.5)